In [ ]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import copy

import random
from itertools import chain, combinations, tee
import time

import math

plt.rcParams['text.usetex'] = True

# Functions

In [ ]:
def demand_name_by_group_index(index):
    list_demand_names = ["Demand (eligible group, 1)", "Demand (eligible group, 2)", \
                         "Demand (ineligible group, 1)", "Demand (ineligible group, 2)", \
                         "Demand (ineligible group, 3)"]
    return list_demand_names[index]

def VoT_name_by_group_index(index):
    list_demand_names = ["VoT (eligible group, 1)", "VoT (eligible group, 2)", \
                         "VoT (ineligible group, 1)", "VoT (ineligible group, 2)", \
                         "VoT (ineligible group, 3)"]
    return list_demand_names[index]

def equals(a, b, tol = 1E-3):
    if abs(a-b) <= tol:
        return True
    else:
        return False
    
def equals_array(arr_1, arr_2, tol = 1E-2):
    if np.linalg.norm(arr_1 - arr_2) <= tol:
        return True
    else:
        return False
    

In [ ]:
def latency_max(flow_max, coeff):
    
    assert np.all(coeff >= 0.0), "coeff should be non-negative"
    assert len(coeff.shape) == 1, "coeff should be a 1-D array."
    assert coeff.shape[0] == 3, "Latency functions are assumed to be piecewise linear / affine with 3 parameters."
    
    return coeff[0] + max(coeff[1] * (flow_max - coeff[2]), 0)

In [ ]:
def round_to_sig_figs(number, sig_figs):
    if number == 0:
        return 0.0
    
    # Calculate the exponent for scientific notation
    exponent = math.floor(math.log10(abs(number)))
    
    # Calculate the scaling factor to bring the number to a single digit before the decimal
    scale = 10**(sig_figs - 1 - exponent)
    
    # Scale, round, and then unscale
    rounded_number = round(number * scale) / scale
    
    return rounded_number

def round_to_sig_figs_array_1D(arr, sig_figs):
    
    arr_rounded = np.zeros(arr.shape)
    
    for i in range(arr.shape[0]):
        arr_rounded[i] = round_to_sig_figs(arr[i], sig_figs)
            
    return arr_rounded

def round_to_sig_figs_array_2D(arr, sig_figs):
    
    arr_rounded = np.zeros(arr.shape)
    
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            arr_rounded[i, j] = round_to_sig_figs(arr[i, j], sig_figs)
            
    return arr_rounded

In [ ]:
# arr_1 = np.array([[1, 2, 3], [4, 5, 6]])
# np.linalg.norm(arr_1)

# Download Groups, Routes to Edges Data:

In [ ]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
# df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_1.csv')
df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_3.csv')

# df_od_flow_data
# df_data

In [ ]:
dict_data = {}

for column_name_full in list(df_data.columns):
    if column_name_full == "Data Category":
        categories_list = df_data[column_name_full].tolist()
    else:
        dict_data[int(column_name_full)] = {}
        for category_index, category in enumerate(categories_list):
            if category == "Start City Index" or category == "End City Index":
                dict_data[int(column_name_full)][category] \
                    = int(df_data[column_name_full].tolist()[category_index])
            elif category == "Start City" or category == "End City":
                dict_data[int(column_name_full)][category] \
                    = df_data[column_name_full].tolist()[category_index]
            else:
#                 print("category:", category)
                dict_data[int(column_name_full)][category] \
                    = float(df_data[column_name_full].tolist()[category_index])

# Test git

In [ ]:
dict_data

In [ ]:
cities_dict = {}
for od_info in list(dict_data.values()):
    if od_info["Start City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["Start City Index"]] = od_info["Start City"]
    if od_info["End City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["End City Index"]] = od_info["End City"]

cities_list = list(cities_dict.values())

# cities_dict

In [ ]:
od_to_edges_array = np.zeros((len(list(dict_data.keys())), 2))

for od_index, od_info in dict_data.items():
    od_to_edges_array[od_index, 0] = int(cities_list.index(od_info["Start City"]))
    od_to_edges_array[od_index, 1] = int(cities_list.index(od_info["End City"]))

edge_to_od_dict = {}
num_edges = int(np.max(od_to_edges_array)) + 1
# print("num_edges:", num_edges)

for e in range(num_edges):
    edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                           if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]
    
# od_to_edges_array
# edge_to_od_dict

In [ ]:
num_groups_per_od = 5

demand_array = np.zeros((len(list(dict_data.keys())), num_groups_per_od))
VoT_array_base = np.zeros((len(list(dict_data.keys())), num_groups_per_od))

for od_index, od_value in dict_data.items():
    for group_index in range(num_groups_per_od):
        demand_name = demand_name_by_group_index(group_index)
        VoT_name = VoT_name_by_group_index(group_index)
        
        demand_array[od_index, group_index] = od_value[demand_name]
        VoT_array_base[od_index, group_index] = od_value[VoT_name]

print(demand_array)
# VoT_array_base

In [ ]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'

T = 5
VoT_array = np.zeros((VoT_array_base.shape[0], VoT_array_base.shape[1], T))

for t in range(T):
    df_perturbation_data = pd.read_csv(directory_path + 'perturbations_1_' + str(t) + '.csv')
    perturbation_array = df_perturbation_data.to_numpy()[:, 1:]
    VoT_array[:, :, t] = VoT_array_base * perturbation_array
    
# VoT_array_base
# perturbation_array

# Download Latency Parameters Data

In [ ]:
directory_path_latency = '../data/pems_latency_inference___101_N_Sep_to_Nov_2024/'
df_latency_params = pd.read_csv(directory_path_latency + 'latency_params.csv')

# list(df_latency_params.loc[:, "Palo Alto"])

In [ ]:
dict_latency_params = {}

city_list = list(df_latency_params.columns)[1:]

for city in city_list:
#     if city != "Belmont":
    if 1 == 1:
        dict_latency_params[city] = {}
        dict_latency_params[city]["Flow (at bend)"] = df_latency_params.loc[:, city][0]
        dict_latency_params[city]["Latency (at bend)"] = df_latency_params.loc[:, city][1]
        dict_latency_params[city]["Slope (after bend)"] = df_latency_params.loc[:, city][2]

dict_latency_params

In [ ]:
# num_edges = 7
num_gp_lanes = 3

num_el = 2
num_groups = demand_array.shape[1]

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

coeff_input = np.zeros((3, num_edges))
for counter, city in enumerate(dict_latency_params.keys()):
    coeff_input[0, counter] = dict_latency_params[city]["Latency (at bend)"]
    coeff_input[1, counter] = dict_latency_params[city]["Slope (after bend)"]
    coeff_input[2, counter] = dict_latency_params[city]["Flow (at bend)"]
    

In [ ]:
## Set lambdas:

lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0

## Initialize tau, alpha values:

filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))

directory_inits = '../data/opt_CBCP_values___' + str(num_el) + '_el_groups/'
# directory_inits = '../data/opt_CBCP_values___' + str(num_el) + '_el_groups___before_20251001/'
df_inits = pd.read_csv(directory_inits + filename_segment + '___tau_B_stats_CBCP.csv')

print("filename_segment:", filename_segment)
print()

inits_tau_arr_as_object = df_inits.to_numpy()[:, 1:6]
inits_B_arr_as_object = df_inits.to_numpy()[0, 7]

argmin_tau = np.zeros((num_edges, T))
argmin_B = 0

for e in range(num_edges):
    for t in range(T):
        argmin_tau[e, t] = inits_tau_arr_as_object[e, t]
        argmin_B = inits_B_arr_as_object

print("argmin_tau:\n", argmin_tau)
print()
print("argmin_B:\n", argmin_B)


In [ ]:
# argmin_tau = np.array([[0.0, 0.3194, 0.3194, 0.0, 0.0], \
#                        [0.2498, 0.2498, 0.2498, 0.0, 0.0], \
#                        [0.0, 0.0, 0.0, 0.9995, 0.9995], \
#                        [0.0, 1.0281, 1.0281, 1.0281, 0.0], \
#                        [1.6043, 0.0, 0.0, 1.6043, 0.0], \
#                        [0.0, 0.1922, 0.1922, 0.1922, 0.0], \
#                        [0.0, 0.0, 0.0, 0.2178, 0.2178]])

# argmin_B = 10.6925

# Compute CBCP and DBCP comparison statistics

In [ ]:
# demand_array
# edge_to_od_dict
# od_to_edges_array

# demand_array[:, 0:3]
# demand_array[:, 3:5]
# demand_array

# np.sum(demand_array[:, 0:3], axis=1).shape

In [ ]:
demand_array_el = np.sum(demand_array[:, 0:3], axis=1)
demand_array_in = np.sum(demand_array[:, 3:5], axis=1)

demand_array_el_across_edges = np.zeros(num_edges)
demand_array_in_across_edges = np.zeros(num_edges)
demand_array_across_edges = np.zeros(num_edges)

for e in range(num_edges):
    demand_array_el_across_edges[e] = np.sum([demand_array_el[od] for od in edge_to_od_dict[e]])
    demand_array_in_across_edges[e] = np.sum([demand_array_in[od] for od in edge_to_od_dict[e]])
    demand_array_across_edges[e] = demand_array_el_across_edges[e] + demand_array_in_across_edges[e]

# demand_array_el_across_edges[e]

In [ ]:
## Verify that demand_array_across_edges = demand_array_el_across_edges + demand_array_in_across_edges

print("demand_array_el_across_edges:", demand_array_el_across_edges)
print("demand_array_in_across_edges:", demand_array_in_across_edges)
print("demand_array_across_edges:", demand_array_across_edges)
print()

demand_array_consistent_tolerance = 1E-2
demand_array_consistent = np.all(np.abs(demand_array_across_edges \
                                        - demand_array_el_across_edges - demand_array_in_across_edges) \
                                <= demand_array_consistent_tolerance)

print(demand_array_consistent)


In [ ]:
# Read data from csv and store into numpy files

directory_CBCP = '../data/opt_CBCP_values___' + str(num_el) + '_el_groups/'
directory_DBCP = '../data/opt_DBCP_values___' + str(num_el) + '_el_groups/'
# directory_CBCP = '../data/opt_CBCP_values___' + str(num_el) + '_el_groups___before_20251001/'
# directory_DBCP = '../data/opt_DBCP_values___' + str(num_el) + '_el_groups___before_20251001/'

tau_CBCP_dict = {}
tau_time_averaged_CBCP_dict = {}
B_CBCP_dict = {}
percent_express_lane_use_CBCP_dict = {}
avg_travel_time_CBCP_dict = {}
total_costs_CBCP_dict = {}

tau_DBCP_dict = {}
tau_time_averaged_DBCP_dict = {}
alpha_DBCP_dict = {}
percent_express_lane_use_DBCP_dict = {}
avg_travel_time_DBCP_dict = {}
total_costs_DBCP_dict = {}


# lambdas_array = np.array([[1.0, 1.0, 1.0], \
#                           [1.0, 5.0, 1.0], \
#                           [1.0, 10.0, 1.0], \
#                           [5.0, 5.0, 1.0], \
#                           [5.0, 10.0, 1.0], \
#                           [10.0, 10.0, 1.0], \
#                           [1.0, 1.0, 0.0], \
#                           [1.0, 5.0, 0.0], \
#                           [5.0, 10.0, 0.0]])

lambdas_array = np.array([[1.0, 1.0, 1.0], \
                          [1.0, 5.0, 1.0], \
                          [1.0, 10.0, 1.0], \
                          [5.0, 5.0, 1.0], \
                          [5.0, 10.0, 1.0], \
                          [10.0, 10.0, 1.0], \
                          [1.0, 1.0, 0.0], \
                          [1.0, 5.0, 0.0], \
                          [5.0, 10.0, 0.0], \
                          [5.0, 1.0, 1.0], \
                          [10.0, 5.0, 1.0], \
                          [10.0, 1.0, 1.0], \
                          [20.0, 1.0, 1.0], \
                          [5.0, 1.0, 0.0], \
                          [10.0, 5.0, 0.0], \
                          [10.0, 1.0, 0.0], \
                          [20.0, 1.0, 0.0], \
                          [5.0, 0.0, 1.0], \
                          [10.0, 0.0, 1.0], \
                          [20.0, 0.0, 1.0], \
                         ])

for lambdas_index in range(lambdas_array.shape[0]):
    lambda_E, lambda_R, lambda_I = lambdas_array[lambdas_index]
    
    filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))
    
    lambda_E, lambda_R, lambda_I = lambdas_array[lambdas_index]
    lambdas = (lambda_E, lambda_R, lambda_I)
    filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))

    # Inputting CBCP data:

    df_CBCP = pd.read_csv(directory_CBCP + filename_segment + '___tau_B_stats_CBCP.csv')
    tau_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 1:6].astype(float)
    tau_time_averaged_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 6].astype(float)
    B_CBCP_dict[lambdas] = df_CBCP.to_numpy()[0, 7]
    percent_express_lane_use_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 8:11].astype(float)
    avg_travel_time_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 11:13].astype(float)
    total_costs_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 13:].astype(float)

    # Inputting DBCP data:

    df_DBCP = pd.read_csv(directory_DBCP + filename_segment + '___tau_alpha_stats_DBCP.csv')
    tau_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 1:6].astype(float)
    tau_time_averaged_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 6].astype(float)
    alpha_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 7:12].astype(float)
    percent_express_lane_use_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 12:15].astype(float)
    avg_travel_time_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 15:17].astype(float)
    total_costs_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 17:].astype(float)
    


In [ ]:
#  total_costs_DBCP_dict[(5.0, 10.0, 1.0)]

df_DBCP

In [ ]:
# Process data into average or total costs, and storing into dict_avg_stats:

dict_avg_stats = {}

for lambdas_index in range(lambdas_array.shape[0]):
    lambda_E, lambda_R, lambda_I = lambdas_array[lambdas_index]
    lambdas = (lambda_E, lambda_R, lambda_I)

    # Processing CBCP data:
    
    dict_index = (lambda_E, lambda_R, lambda_I, 'CBCP')
    
    dict_avg_stats[dict_index] = {}
    
    dict_avg_stats[dict_index]['percent of overall'] \
        = np.sum(percent_express_lane_use_CBCP_dict[lambdas][:, 0] * demand_array_across_edges) \
            / np.sum(demand_array_across_edges)
    dict_avg_stats[dict_index]['percent of eligible'] \
        = np.sum(percent_express_lane_use_CBCP_dict[lambdas][:, 1] * demand_array_el_across_edges) \
            / np.sum(demand_array_el_across_edges)
    dict_avg_stats[dict_index]['percent of ineligible'] \
        = np.sum(percent_express_lane_use_CBCP_dict[lambdas][:, 2] * demand_array_in_across_edges) \
            / np.sum(demand_array_in_across_edges)
    
    dict_avg_stats[dict_index]['average travel time, express'] \
        = np.sum(avg_travel_time_CBCP_dict[lambdas][:, 0])
    dict_avg_stats[dict_index]['average travel time, general purpose'] \
        = np.sum(avg_travel_time_CBCP_dict[lambdas][:, 1])
    
    dict_avg_stats[dict_index]['total travel cost, eligible'] \
        = np.sum(total_costs_CBCP_dict[lambdas][:, 0])
    dict_avg_stats[dict_index]['total travel cost, ineligible'] \
        = np.sum(total_costs_CBCP_dict[lambdas][:, 1])
    dict_avg_stats[dict_index]['total toll revenue'] \
        = np.sum(total_costs_CBCP_dict[lambdas][:, 2])
    dict_avg_stats[dict_index]['total societal cost'] \
        = np.sum(total_costs_CBCP_dict[lambdas][:, 3])

#     # Processing DBCP data:
    
    dict_index = (lambda_E, lambda_R, lambda_I, 'DBCP')
    
    dict_avg_stats[dict_index] = {}
    
    dict_avg_stats[dict_index]['percent of overall'] \
        = np.sum(percent_express_lane_use_DBCP_dict[lambdas][:, 0] * demand_array_across_edges) \
            / np.sum(demand_array_across_edges)
    dict_avg_stats[dict_index]['percent of eligible'] \
        = np.sum(percent_express_lane_use_DBCP_dict[lambdas][:, 1] * demand_array_el_across_edges) \
            / np.sum(demand_array_el_across_edges)
    dict_avg_stats[dict_index]['percent of ineligible'] \
        = np.sum(percent_express_lane_use_DBCP_dict[lambdas][:, 2] * demand_array_in_across_edges) \
            / np.sum(demand_array_in_across_edges)
    
    dict_avg_stats[dict_index]['average travel time, express'] \
        = np.sum(avg_travel_time_DBCP_dict[lambdas][:, 0])
    dict_avg_stats[dict_index]['average travel time, general purpose'] \
        = np.sum(avg_travel_time_DBCP_dict[lambdas][:, 1])
    
    dict_avg_stats[dict_index]['total travel cost, eligible'] \
        = np.sum(total_costs_DBCP_dict[lambdas][:, 0])
    dict_avg_stats[dict_index]['total travel cost, ineligible'] \
        = np.sum(total_costs_DBCP_dict[lambdas][:, 1])
    dict_avg_stats[dict_index]['total toll revenue'] \
        = np.sum(total_costs_DBCP_dict[lambdas][:, 2])
    dict_avg_stats[dict_index]['total societal cost'] \
        = np.sum(total_costs_DBCP_dict[lambdas][:, 3])
    

In [ ]:
# lambdas = (1.0, 1.0, 1.0)

# percent_express_lane_use_CBCP_dict[lambdas][:, 0]

# np.sum(percent_express_lane_use_CBCP_dict[lambdas][:, 0] * demand_array_across_edges) \
#             / np.sum(demand_array_across_edges)

# dict_avg_stats[(1.0, 1.0, 1.0, 'CBCP')]

In [ ]:
avg_stats_array = np.zeros((lambdas_array.shape[0] * 2, 13))

for lambdas_index in range(lambdas_array.shape[0]):
    lambda_E, lambda_R, lambda_I = lambdas_array[lambdas_index]
    
    # Storing CBCP avg stats as array:
    
    dict_index = (lambda_E, lambda_R, lambda_I, 'CBCP')
    
    avg_stats_array[2 * lambdas_index, 0] = lambda_E
    avg_stats_array[2 * lambdas_index, 1] = lambda_R
    avg_stats_array[2 * lambdas_index, 2] = lambda_I
    
    # Here, "0" indicates CBCP
    avg_stats_array[2 * lambdas_index, 3] = 0
    
    avg_stats_array[2 * lambdas_index, 4] = dict_avg_stats[dict_index]['percent of overall']
    avg_stats_array[2 * lambdas_index, 5] = dict_avg_stats[dict_index]['percent of eligible']
    avg_stats_array[2 * lambdas_index, 6] = dict_avg_stats[dict_index]['percent of ineligible']
    
    avg_stats_array[2 * lambdas_index, 7] = dict_avg_stats[dict_index]['average travel time, express']
    avg_stats_array[2 * lambdas_index, 8] = dict_avg_stats[dict_index]['average travel time, general purpose']

    avg_stats_array[2 * lambdas_index, 9] = dict_avg_stats[dict_index]['total travel cost, eligible']
    avg_stats_array[2 * lambdas_index, 10] = dict_avg_stats[dict_index]['total travel cost, ineligible']
    avg_stats_array[2 * lambdas_index, 11] = dict_avg_stats[dict_index]['total toll revenue']
    avg_stats_array[2 * lambdas_index, 12] = dict_avg_stats[dict_index]['total societal cost']
    
    # Storing DBCP avg stats as array:
    
    dict_index = (lambda_E, lambda_R, lambda_I, 'DBCP')
    
    avg_stats_array[2 * lambdas_index + 1, 0] = lambda_E
    avg_stats_array[2 * lambdas_index + 1, 1] = lambda_R
    avg_stats_array[2 * lambdas_index + 1, 2] = lambda_I
    
    # Here, "1" indicates DBCP
    avg_stats_array[2 * lambdas_index + 1, 3] = 1
    
    avg_stats_array[2 * lambdas_index + 1, 4] = dict_avg_stats[dict_index]['percent of overall']
    avg_stats_array[2 * lambdas_index + 1, 5] = dict_avg_stats[dict_index]['percent of eligible']
    avg_stats_array[2 * lambdas_index + 1, 6] = dict_avg_stats[dict_index]['percent of ineligible']
    
    avg_stats_array[2 * lambdas_index + 1, 7] = dict_avg_stats[dict_index]['average travel time, express']
    avg_stats_array[2 * lambdas_index + 1, 8] = dict_avg_stats[dict_index]['average travel time, general purpose']

    avg_stats_array[2 * lambdas_index + 1, 9] = dict_avg_stats[dict_index]['total travel cost, eligible']
    avg_stats_array[2 * lambdas_index + 1, 10] = dict_avg_stats[dict_index]['total travel cost, ineligible']
    avg_stats_array[2 * lambdas_index + 1, 11] = dict_avg_stats[dict_index]['total toll revenue']
    avg_stats_array[2 * lambdas_index + 1, 12] = dict_avg_stats[dict_index]['total societal cost']


In [ ]:
avg_stats_array.shape

In [ ]:
avg_stats_array_rounded = np.zeros(avg_stats_array.shape)

for lambdas_index in range(lambdas_array.shape[0]):
    lambda_E, lambda_R, lambda_I = lambdas_array[lambdas_index]
    
    # Storing CBCP avg stats as array:
    
    dict_index = (lambda_E, lambda_R, lambda_I, 'CBCP')
    
    avg_stats_array_rounded[2 * lambdas_index, 0] = lambda_E
    avg_stats_array_rounded[2 * lambdas_index, 1] = lambda_R
    avg_stats_array_rounded[2 * lambdas_index, 2] = lambda_I
    
    # Here, "0" indicates CBCP
    avg_stats_array_rounded[2 * lambdas_index, 3] = 0
    
    avg_stats_array_rounded[2 * lambdas_index, 4] = np.round(avg_stats_array[2 * lambdas_index, 4], 2)
    avg_stats_array_rounded[2 * lambdas_index, 5] = np.round(avg_stats_array[2 * lambdas_index, 5], 2)
    avg_stats_array_rounded[2 * lambdas_index, 6] = np.round(avg_stats_array[2 * lambdas_index, 6], 2)
    
    avg_stats_array_rounded[2 * lambdas_index, 7] = avg_stats_array[2 * lambdas_index, 7]
    avg_stats_array_rounded[2 * lambdas_index, 8] = avg_stats_array[2 * lambdas_index, 8]

    avg_stats_array_rounded[2 * lambdas_index, 9] = round_to_sig_figs(avg_stats_array[2 * lambdas_index, 9], 3)
    avg_stats_array_rounded[2 * lambdas_index, 10] = round_to_sig_figs(avg_stats_array[2 * lambdas_index, 10], 3)
    avg_stats_array_rounded[2 * lambdas_index, 11] = round_to_sig_figs(avg_stats_array[2 * lambdas_index, 11], 3)
    avg_stats_array_rounded[2 * lambdas_index, 12] = round_to_sig_figs(avg_stats_array[2 * lambdas_index, 12], 3)
    
    # Storing DBCP avg stats as array:
    
    dict_index = (lambda_E, lambda_R, lambda_I, 'DBCP')
    
    avg_stats_array_rounded[2 * lambdas_index + 1, 0] = lambda_E
    avg_stats_array_rounded[2 * lambdas_index + 1, 1] = lambda_R
    avg_stats_array_rounded[2 * lambdas_index + 1, 2] = lambda_I
    
    # Here, "1" indicates DBCP
    avg_stats_array_rounded[2 * lambdas_index + 1, 3] = 1
    
    avg_stats_array_rounded[2 * lambdas_index + 1, 4] = np.round(avg_stats_array[2 * lambdas_index + 1, 4], 2)
    avg_stats_array_rounded[2 * lambdas_index + 1, 5] = np.round(avg_stats_array[2 * lambdas_index + 1, 5], 2)
    avg_stats_array_rounded[2 * lambdas_index + 1, 6] = np.round(avg_stats_array[2 * lambdas_index + 1, 6], 2)
    
    avg_stats_array_rounded[2 * lambdas_index + 1, 7] = avg_stats_array[2 * lambdas_index + 1, 7]
    avg_stats_array_rounded[2 * lambdas_index + 1, 8] = avg_stats_array[2 * lambdas_index + 1, 8]

    avg_stats_array_rounded[2 * lambdas_index + 1, 9] = round_to_sig_figs(avg_stats_array[2 * lambdas_index + 1, 9], 3)
    avg_stats_array_rounded[2 * lambdas_index + 1, 10] = round_to_sig_figs(avg_stats_array[2 * lambdas_index + 1, 10], 3)
    avg_stats_array_rounded[2 * lambdas_index + 1, 11] = round_to_sig_figs(avg_stats_array[2 * lambdas_index + 1, 11], 3)
    avg_stats_array_rounded[2 * lambdas_index + 1, 12] = round_to_sig_figs(avg_stats_array[2 * lambdas_index + 1, 12], 3)
    

In [ ]:
# np.all(avg_stats_array_rounded == avg_stats_array)

In [ ]:
column_names = []
column_names += ["lambda_E", "lambda_R", "lambda_I"]
column_names += ["CBCP (=0) or DBCP (=1)"]
column_names += ["% overall users using express lanes", \
                 "% eligible users using express lanes", \
                 "% ineligible users using express lanes", \
                 "Average travel time (express lanes)", \
                 "Average travel time (general purpose lanes)", \
                 "Total travel cost (eligible users)", \
                 "Total travel cost (ineligible users)", \
                 "Total toll revenue", \
                 "Total societal cost"]

df_avg_stats_to_save = pd.DataFrame(avg_stats_array_rounded, index=None, columns=column_names)

df_avg_stats_to_save

In [ ]:
directory_to_save = "../data/stats_compare___" + str(num_el) + "_el_groups/"
# directory_to_save = "../data/stats_compare___" + str(num_el) + "_el_groups___before_20251001/"

# filename = "opt_CBCP_params___" + random_string + '.csv'

filename = 'avg_stats___' + str(num_el) + '_el_groups.csv'

df_avg_stats_to_save.to_csv(directory_to_save + filename)

In [ ]:
# # Stats array are: DBCP minus CBCP

# stats_compare_percentage_array = np.zeros((lambdas_array.shape[0], 11))

# for lambdas_index in range(lambdas_array.shape[0]):
# # for lambdas_index in [12]:
#     print()
#     print("lambdas_index:", lambdas_index)
#     print("avg_stats_array[2 * lambdas_index, 4:12]:\n", avg_stats_array[2 * lambdas_index, 4:12])
    
#     stats_compare_percentage_array[lambdas_index, 0:3] = lambdas_array[lambdas_index, 0:3]
    
#     stats_compare_percentage_array[lambdas_index, 3:] = \
#         100 * (avg_stats_array[2 * lambdas_index + 1, 4:12] - avg_stats_array[2 * lambdas_index, 4:12]) \
#             / avg_stats_array[2 * lambdas_index, 4:12]



In [ ]:
# # stats_compare_percentage_array
# lambdas_array

In [ ]:
# Stats array are: DBCP minus CBCP

## Percentage difference:

# stats_compare_percentage_array = np.zeros((lambdas_array.shape[0], 12))

# for lambdas_index in range(lambdas_array.shape[0]):
#     print()
#     print("lambdas_index:", lambdas_index)
#     print("avg_stats_array[2 * lambdas_index, 4:13]:\n", avg_stats_array[2 * lambdas_index, 4:13])
    
#     stats_compare_percentage_array[lambdas_index, 0:3] = lambdas_array[lambdas_index, 0:3]
    
#     stats_compare_percentage_array[lambdas_index, 3:] = \
#         100 * (avg_stats_array[2 * lambdas_index + 1, 4:13] - avg_stats_array[2 * lambdas_index, 4:13]) \
#             / np.abs(avg_stats_array[2 * lambdas_index, 4:13])


stats_compare_absolute_percentage_array = np.zeros((lambdas_array.shape[0]*2, 13))

for lambdas_index in range(lambdas_array.shape[0]):
    print()
    print("lambdas_index:", lambdas_index)
    print("avg_stats_array[2 * lambdas_index, 4:13]:\n", avg_stats_array[2 * lambdas_index, 4:13])
    
    stats_compare_absolute_percentage_array[2 * lambdas_index, 0:3] \
        = np.round(lambdas_array[lambdas_index, 0:3], decimals=2)
    stats_compare_absolute_percentage_array[2 * lambdas_index, 3] = 0
#     stats_compare_absolute_percentage_array[2 * lambdas_index, 4:13] = \
#         avg_stats_array[2 * lambdas_index + 1, 4:13] - avg_stats_array[2 * lambdas_index, 4:13]
    stats_compare_absolute_percentage_array[2 * lambdas_index, 4:10] \
        = np.round(avg_stats_array[2 * lambdas_index + 1, 4:10] - avg_stats_array[2 * lambdas_index, 4:10], decimals=2)
    stats_compare_absolute_percentage_array[2 * lambdas_index, 10:13] \
        = round_to_sig_figs_array_1D(avg_stats_array[2 * lambdas_index + 1, 10:13] - avg_stats_array[2 * lambdas_index, 10:13], 3)
    
    stats_compare_absolute_percentage_array[2 * lambdas_index + 1, 0:3] \
        = np.round(lambdas_array[lambdas_index, 0:3], decimals=2)
    stats_compare_absolute_percentage_array[2 * lambdas_index + 1, 3] = 1
    stats_compare_absolute_percentage_array[2 * lambdas_index + 1, 4:] \
        = np.round(100 * (avg_stats_array[2 * lambdas_index + 1, 4:13] - avg_stats_array[2 * lambdas_index, 4:13]) \
                 / np.abs(avg_stats_array[2 * lambdas_index, 4:13]), decimals=2)





In [ ]:
column_names = []
column_names += ["lambda_E", "lambda_R", "lambda_I"]
column_names += ["Absolute (=0) or Relative (=1) Difference"]
column_names += ["Diff, % overall users using express lanes", \
                 "Diff, % eligible users using express lanes", \
                 "Diff, % ineligible users using express lanes", \
                 "Diff, Average travel time (express lanes)", \
                 "Diff, Average travel time (general purpose lanes)", \
                 "Diff, Total travel cost (eligible users)", \
                 "Diff, Total travel cost (ineligible users)", \
                 "Diff, Total toll revenue", \
                 "Diff, Total societal cost"]

df_stats_compare_to_save = pd.DataFrame(stats_compare_absolute_percentage_array, index=None, columns=column_names)

df_stats_compare_to_save

In [ ]:
# column_names = []
# column_names += ["lambda_E", "lambda_R", "lambda_I"]
# column_names += ["% overall users using express lanes", \
#                  "% eligible users using express lanes", \
#                  "% ineligible users using express lanes", \
#                  "Average travel time (express lanes)", \
#                  "Average travel time (general purpose lanes)", \
#                  "Total travel cost (eligible users)", \
#                  "Total travel cost (ineligible users)", \
#                  "Total toll revenue"]

# df_stats_compare_to_save = pd.DataFrame(stats_compare_percentage_array, index=None, columns=column_names)

# df_stats_compare_to_save

In [ ]:
# directory_to_save = "../data/stats_compare/"
# filename = "opt_CBCP_params___" + random_string + '.csv'

filename = 'compare_stats___' + str(num_el) + '_el_groups.csv'

df_stats_compare_to_save.to_csv(directory_to_save + filename)

## <font color='red'>STOP. End of Code to run.</font> 

# End of code to run.

In [ ]:
# np.sum(np.array([1, 2, 3]) * np.array([5, 3, 1]))


In [ ]:
# avg_travel_time_CBCP_dict[(1.0, 1.0, 1.0)]

In [ ]:
# tau_CBCP_dict = {}
# tau_time_averaged_CBCP_dict = {}
# B_CBCP_dict = {}
# percent_express_lane_use_CBCP_dict = {}
# avg_travel_time_CBCP_dict = {}
# total_costs_CBCP_dict = {}

# tau_DBCP_dict = {}
# tau_time_averaged_DBCP_dict = {}
# alpha_DBCP_dict = {}
# percent_express_lane_use_DBCP_dict = {}
# avg_travel_time_DBCP_dict = {}
# total_costs_DBCP_dict = {}


# directory_CBCP = '../data/opt_CBCP_values___' + str(num_el) + '_el_groups/'
# directory_DBCP = '../data/opt_DBCP_values___' + str(num_el) + '_el_groups/'

# lambdas_index = 4
# lambda_E, lambda_R, lambda_I = lambdas_array[lambdas_index]
# lambdas = (lambda_E, lambda_R, lambda_I)
# filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))

# # Inputting CBCP data:

# df_CBCP = pd.read_csv(directory_CBCP + filename_segment + '___tau_B_stats_CBCP.csv')
# tau_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 1:6]
# tau_time_averaged_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 6]
# B_CBCP_dict[lambdas] = df_CBCP.to_numpy()[0, 7]
# percent_express_lane_use_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 8:11]
# avg_travel_time_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 11:12]
# total_costs_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 12:]

# # Inputting DBCP data:

# df_DBCP = pd.read_csv(directory_DBCP + filename_segment + '___tau_alpha_stats_DBCP.csv')
# tau_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 1:6]
# tau_time_averaged_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 6]
# alpha_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 7:12]
# percent_express_lane_use_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 12:15]
# avg_travel_time_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 15:16]
# total_costs_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 16:]



## Test:

## <font color='red'>Colored Font Titles</font> 

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


## Linear Approximation for Latency Function:

In [ ]:
# Variables:
v = cp.Variable(1)
            
# Objective:
func = v - 1 + cp.square(cp.maximum(v-1, 0))
objective = cp.Minimize(func)

# Constraints:
constraints = [-3.0 <= v, v <= 3.0]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("v.value:", v.value)
